We begin with a simple plot to give us a starting point and validate that our simulation is giving sensible results. We would expect that significantly below the critical value, all of the percolation clusters in our simulation would terminate, whilst significantly above the critical value there would be one single cluster which dominates, and does not terminate. A simple plot of log cluster size against log number of clusters over a range of probabilities shows this clearly.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

def read_data_dir(directory):
  data = []
  probabilities = []
  nums_samples = []

  for filename in sorted(os.listdir(directory)):
    path = os.path.join(directory, filename)

    with open(path) as fp:
      for i, line in enumerate(fp):
          if i == 1:
            params = line.split(',')
            probabilities.append(float(params[0]))
            nums_samples.append(float(params[3])*float(params[1])**3)
            break

    data.append(np.genfromtxt(path, delimiter=',', skip_header=4))

  return (probabilities, nums_samples, data)

def plot_size_num_prob_3d(directory):
  x1data = [] # terminated clusters
  x2data = [] # non-terminated clusters
  ydata = [] # probability
  zdata = [] # size

  probabilities, nums_samples, data = read_data_dir(directory)

  for i, d in enumerate(data):
    x1data += list(np.log2(d[:,1]))
    x2data += list(np.log2(d[:,2]))
    zdata += list(np.log2(d[:,0]))
    ydata += [probabilities[i] for _ in d[:,0]]

  fig = plt.figure()
  fig.set_size_inches((8, 8))
  ax = fig.add_subplot(projection='3d')
  ax.scatter(x1data, ydata, zdata, label="terminated")
  ax.scatter(x2data, ydata, zdata, label="still growing")
  ax.set_xlabel("log2 num_clusters")
  ax.set_ylabel("probability")
  ax.set_zlabel("log2 size")
  ax.legend()
  ax.set_box_aspect(None, zoom=0.9) # Work around for matplotlib bug which cuts off z label

plot_size_num_prob_3d("data/sanity_check")


Let $n_s(p)$ be the average number of clusters per lattice point of size $s$ for a given probability $p$. According to standard scaling theory (e.g. [MM]) we have that

$$n_s(p) = s^{-\tau}(f_0(z)+s^{-\Omega}f_1(z)+...)$$

where $\tau$ is the _fisher exponent_, $\Omega$ accounts for the leading errors due to the finite size of our lattices, $z=(p-p_c)s^{\sigma}$ for some constant $\sigma$ and $f_0, f_1...$ are analytic in a neighbourhood of zero. Taking the taylor expansion of the $f_i$ we obtain

$$n_s(p) = s^{-\tau}(c_0 + c_1(p-p_c)s^{\sigma} + c_2s^{-\Omega} + c_3(p-p_c)s^{\sigma - \Omega} + ...)$$

Taking logs, pulling out a constant factor and taking the Taylor expansion of the $f_i$, we obtain

$$
\begin{aligned}
\log(n_s) &= -\tau \log(s) + a_0 + \log\left(1 + a_1(p-p_c)s^{\sigma} + a_2s^{-\Omega} + a_3(p-p_c)s^{\sigma - \Omega} + ...\right) \\
&= -\tau \log(s) + a_0 + a_1(p-p_c)s^{\sigma} + a_2s^{-\Omega} + a_3(p-p_c)s^{\sigma - \Omega} + ...
\end{aligned}
$$

in a neighbourhood of $p_c$. Thus (aside from the effects of $\Omega$) we expect the critical value to occur when we have a log-linear relationship between $n_s$ and $s$. This aligns well with our initial sanity-check plot.

It will be convenient for our simulations to sample individual points from our lattice, rather than counting clusters. So we instead consider the probability $P(s)$ that a given point lives in a cluster of size $s$, which is simply $sn_s$.

One obstacle we must overcome if we wish to make accurate predictions is the fact that our simulations must always be finite. In order to remove boundary effects, it is convenient to consider the cumulative distribution $Q(s) = \sum_{t=s}^{\infty}P(t)$. This way, we can simply collect up all the terms affected by the boundary into the final bin. Approximating this as an integral, we obtain

$$
\begin{aligned}
Q(s,p) = Q(s) &= \int_{s}^{\infty} t^{1-\tau}(c_0 + c_1(p-p_c)t^{\sigma} + c_2t^{-\Omega} + c_3(p-p_c)t^{\sigma - \Omega} + ...)dt + ... \\
&= s^{2-\tau}(c_0 + c_1(p-p_c)s^{\sigma} + c_2s^{-\Omega} + c_3(p-p_c)s^{\sigma - \Omega} + ...) + ...
\end{aligned}
$$

(note that we have absorbed some constant factors into the $c_i$ when evaluating the integral) and so

$$
\begin{equation}
\tag{1}
\log Q(s) = (2-\tau) \log(s) + a_0 + a_1(p-p_c)s^{\sigma} + a_2s^{-\Omega} + a_3(p-p_c)s^{\sigma - \Omega} + ...
\end{equation}
$$

Thus to obtain accurate estimates for $p_c$ (and $\tau$) it remains to generate as much data as possible on the relationship between $n_s$ and $s$, whilst minimising the effects of finite lattice sizes.

We shall therefore sample the size of the parent cluster for all of the points from a smaller central cube of side length $L'$ from our lattice of side length $L$, and sum up our results over a large number of runs.

First, let us zoom in a bit from out initial sanity check plot and obtain a sensible starting point to look for $p_c$.

In [ ]:
probabilities, nums_samples, simulation_data = read_data_dir("data/test1")

for idx, p, num_samples, data in zip(range(len(probabilities)), probabilities, nums_samples, simulation_data):
  size_data = [d for i, d in enumerate(data[:, 0]) if data[i, 2] == 0 and data[i,1] != 0]
  number_data = [np.log2(np.sum(data[i:, 1] + data[i:, 2])/num_samples) for i, d in enumerate(data[:, 1]) if data[i, 2] == 0 and data[i,1] != 0]

  plt.scatter(size_data, number_data, label=f"p={p}")
  plt.xlabel(r"$\log_2s$")
  plt.ylabel(r"$\log_2Q(s)$")

plt.legend()


From this plot we can clearly see that $p_c$ should be somewhere in the region of 0.249, as we are close to linear here as we would expect - probably slightly below this value as $p=0.248$ is somewhat closer to linear than $p=0.25$.

In [ ]:
from scipy.optimize import curve_fit

def rhs(xy, p_c, tau, omega, sigma, a_0, a_1, a_2, a_3):
  lg2_s, p = xy
  return (-(tau - 2) * lg2_s + a_0 + a_1 * (p-p_c) * 2**(lg2_s * sigma) + a_2 * 2**(-lg2_s * omega) + a_3*(p-p_c) * 2**(lg2_s * (sigma - omega)))

probabilities, nums_samples, simulation_data = read_data_dir("data/test2")

size_data = []
p_data = []
lhs_data = []

fig, axs = plt.subplots(4,2)
fig.set_figheight(20)
fig.set_figwidth(16)

for idx, p, num_samples, data in zip(range(len(probabilities)), probabilities, nums_samples, simulation_data):
  size_data += [d for i, d in enumerate(data[:, 0]) if data[i, 2] == 0 and data[i,1] != 0]
  p_data += [p for i, d in enumerate(data[:, 0]) if data[i, 2] == 0 and data[i,1] != 0]
  lhs_data += [np.log2(np.sum(data[i:, 1] + data[i:, 2])/num_samples) for i, d in enumerate(data[:, 1]) if data[i, 2] == 0 and data[i,1] != 0]


popt, pcov = curve_fit(rhs, (size_data, p_data), lhs_data, p0=(0.2488,2.19,0.5,0.5,20,10,10,10), bounds=((0.2, 1, 0.2, 0.2, -100, -100, -100, -100), (0.3, 3, 5, 5, 100, 100, 100, 100)), maxfev=10000)
stddevs = np.sqrt(np.diag(pcov))

param_names = ["p_c", "tau", "omega", "sigma", "a_0", "a_1", "a_2", "a_3"]
for name, value, stddev in zip(param_names, popt, np.sqrt(np.diag(pcov))):
  print(f"{name: <5} = {value: <25} std dev = {stddev}")

# Save these for later
tau = popt[1]
omega = popt[2]
sigma = popt[3]

for idx, p, data in zip(range(len(probabilities)), probabilities, simulation_data):
  size_data = [d for i, d in enumerate(data[:, 0]) if data[i, 2] == 0  and data[i,1] != 0]
  number_data = [np.log2(np.sum(data[i:, 1] + data[i:, 2])/num_samples) for i, d in enumerate(data[:, 1]) if data[i, 2] == 0 and data[i,1] != 0]
  axs[idx//2, idx%2].scatter(size_data, number_data)

  pred_num_data = [rhs((s, p), *popt) for s in size_data]
  axs[idx//2, idx%2].plot(size_data, pred_num_data)

  axs[idx//2, idx%2].set_xlabel("log2 s")
  axs[idx//2, idx%2].set_title(f"p={p}")
  axs[idx//2, idx%2].set_ylabel("log2 Q(s)")


From these fits we can obtain some initial estimates for $p_c$, $\tau$, $\Omega$ and $\sigma$.

In order to estimate for $p_c$ as accurately as possible, it is helpful to now remove the main term from (1). With this in mind, let $R(s)=s^{\tau-2}Q(s)$. Then

$$
\log R(s) = a_0 + a_1(p-p_c)s^{\sigma} + a_2s^{-\Omega} + a_3(p-p_c)s^{\sigma - \Omega} + ...
$$

Differentiating with respect to $s^{\sigma}$ (as used in [LZ]) gives

$$
\frac{\partial\log R(s)}{\partial s^{\sigma}} = a_1(p-p_c) - \frac{\Omega a_2}{\sigma}s^{-\sigma - \Omega} + (1-\frac{\Omega}{\sigma})a_3(p-p_c)s^{-\Omega} + ... \tag{2}
$$

And so when close to $p_c$ for large $s$, we should obtain a linear relationship between $\frac{\partial\log R(s)}{\partial s^{\sigma}}$ and $p$, with the $p$-intercept giving $p_c$.

In [ ]:
probabilities, nums_samples, simulation_data = read_data_dir("data/test2")

fig = plt.gcf()
fig.set_size_inches(12, 8)

for idx, p, num_samples, data in zip(range(len(probabilities)), probabilities, nums_samples, simulation_data):
  size_data = [2**(sigma*d) for i, d in enumerate(data[:, 0]) if data[i, 2] < 100000 and data[i,1] != 0]
  r_data = [np.log2(np.sum(2**(i*(tau-2))*(data[i:, 1] + data[i:, 2]))/num_samples) for i, d in enumerate(data[:, 0]) if data[i, 2] < 100000 and data[i,1] != 0]
  
  plt.scatter(size_data, r_data, label=f"p={p}")
  plt.xlabel(r"$s^{\sigma}$")
  plt.ylabel(r"$\log_2R(s)$")
  plt.legend(loc="lower right")


In [ ]:
probabilities, nums_samples, simulation_data = read_data_dir("data/test2")

def line(x, gradient, intercept):
  return gradient*x + intercept

pdata = []
ydata = []

for idx, p, num_samples, data in zip(range(len(probabilities)), probabilities, nums_samples, simulation_data):
  size_data = [2**(sigma*d) for i, d in enumerate(data[:, 0]) if data[i, 2] == 0 and data[i,1] != 0]
  r_data = [np.log2(np.sum(2**(i*(tau-2))*(data[i:, 1] + data[i:, 2]))/num_samples) for i, d in enumerate(data[:, 0]) if data[i, 2] == 0 and data[i,1] != 0]

  popt, pcov = curve_fit(line, size_data[-6:], r_data[-6:])

  print(f"p={p: <8} gradient={popt[0]: <25} std dev={np.sqrt(np.diag(pcov))[0]}")

  ydata.append(popt[0])
  pdata.append(p)

plt.plot(pdata, ydata)
plt.scatter(pdata, ydata)
plt.xlabel(r"$p$")
plt.ylabel(r"$\frac{\partial\log R(s)}{\partial s^{\sigma}}$")

From this it looks like the critical value must be around the $p=0.2488$ mark. The (approximate) linearity of the plot gives us some confidence that our model in (2) is correct. I will return to this in the near future with _slightly_ more powerful hardware to obtain a more precise estimate. Note that all of the above simulations have been run on WSL running on my 6 year old budget laptop (with 0% battery health) with only 4GB of available RAM and 4 mediocre CPU cores... So it should be possible to obtain much better results with some actual compute and memory.

[MM] S. Mertens and C. Moore, Phys. Rev. E 98, 022120 (2018)

[LZ] C.D. Lorenz and R.M. Ziff, Phys. Rev. E 57, 230 (1998)